# LSTM -Tweet Interarrival Times

This notebook trains an LSTM classifier to detect bot accounts based on the interarrival times between their tweets.

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, 
                             roc_curve, auc)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import seaborn as sns
import warnings


import ijson
import csv
import os
import gc

from torch.nn.utils.rnn import pad_sequence

from sklearn.metrics import (confusion_matrix, roc_curve, auc, 
                             accuracy_score, precision_score, recall_score, f1_score, roc_auc_score)



warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Load Tweet Data from JSON Files

Load tweet data from tweet0.json through tweet8.json files, extracting tweets, labels, and splits.

In [ ]:
# Config
SCRATCH_DIR = "./scratch_temp"
os.makedirs(SCRATCH_DIR, exist_ok=True)
OUTPUT_FILE = os.path.join(SCRATCH_DIR, "processed_timestamps.csv")
DATA_DIR = '../data/twibot22/'

print(f"Running ijson extractor, output: {OUTPUT_FILE}")
print("Loading labels...")
df_labels = pd.read_csv(os.path.join(DATA_DIR, 'label.csv'))

# Remove 'u' from user id value
valid_users = set(df_labels['id'].astype(str).str.replace('u', '', regex=False))

print(f"number of users: {len(valid_users)}.")

del df_labels
gc.collect()

# CSV setup 
with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['user_id', 'timestamp'])

    # Iterate over the tweet_x.json files
    for i in range(9):
        file_path = os.path.join(DATA_DIR, f'tweet_{i}.json')
        print(f"\nWorking with {file_path}...")
        
        try:
            with open(file_path, 'rb') as f_in:
                parser = ijson.items(f_in, 'item')
                
                count = 0
                saved = 0
                
                for tweet in parser:
                    # convert str id to int
                    uid = str(tweet.get('author_id', ''))
                    
                    # Create rows
                    if uid in valid_users:
                        created_at = tweet.get('created_at')
                        if created_at:
                            writer.writerow([uid, created_at])
                            saved += 1
                    
                    count += 1
                    if count % 2000000 == 0:
                        # Control prints
                        print(f"   -> finished {count} tweets... (saved: {saved})")
                        
            print(f" file {i} Fil=nished. Saved in total: {saved}")
            
        except FileNotFoundError:
            print(f" Filer {file_path} does not exitst.")
        except Exception as e:
            print(f"Error in file {file_path}: {e}")

print(f"\n Done, file: {OUTPUT_FILE}")

In [3]:
# config
CSV_PATH = "./scratch_temp/processed_timestamps.csv"
LABEL_PATH = '../data/twibot22/label.csv'
SPLIT_PATH = '../data/twibot22/split.csv'

print("Loading prepared CSV...")

df = pd.read_csv(CSV_PATH, dtype={'user_id': str}) 

print(f"Loaded {len(df)} records. Converting...")

# str -> datetime conversion
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 2. conversion to unix timestamp
# .astype('int64') outputs ns, division by 10^9 gives us s
df['timestamp'] = df['timestamp'].astype('int64') // 10**9

print("Sorting values by time..")
df.sort_values(by=['user_id', 'timestamp'], inplace=True)

print("getting Inter-arrival times...")

df['prev_time'] = df.groupby('user_id')['timestamp'].shift(1)
df['iat'] = df['timestamp'] - df['prev_time']

# delete 1st tweet in the series / invalid ones
df = df.dropna(subset=['iat'])
df = df[df['iat'] > 0]

# Log normalisation ? todo
df['iat_log'] = np.log1p(df['iat'])

print("Creating sequences for each unique user...")
sequences_map = df.groupby('user_id')['iat_log'].apply(list).to_dict()

# Ram cleanup
del df
gc.collect()

print(f"Finished, sequences created for {len(sequences_map)} users.")

Loading prepared CSV...
Loaded 88217457 records. Converting...
Sorting values by time..
getting Inter-arrival times...
Creating sequences for each unique user...
Finished, sequences created for 921037 users.


In [ ]:
print("Loading split and labels...")

df_labels = pd.read_csv(LABEL_PATH)
df_labels['id'] = df_labels['id'].astype(str).str.replace('u', '', regex=False)
label_map = dict(zip(df_labels['id'], (df_labels['label'] == 'bot').astype(int)))

df_split = pd.read_csv(SPLIT_PATH)
df_split['id'] = df_split['id'].astype(str).str.replace('u', '', regex=False)
split_map = dict(zip(df_split['id'], df_split['split']))

# set sequnce len
MAX_SEQ_LEN = 20 

def create_dataset(split_name):
    X_list = []
    y_list = []
    
    # Loop trough users
    for uid, seq in sequences_map.items():
        # Check if user land in set
        if split_map.get(uid) != split_name:
            continue
            
        if uid not in label_map:
            continue
            
        # cut or pad to MAX_SEQ_LEN
        # latest values
        if len(seq) > MAX_SEQ_LEN:
            seq = seq[-MAX_SEQ_LEN:]
        
        # convert to tnesor
        tensor_seq = torch.tensor(seq, dtype=torch.float32)
        
        X_list.append(tensor_seq)
        y_list.append(label_map[uid])
        
    if not X_list:
        print(f"Warning '{split_name}' is empty!")
        return None, None

    # pad with zeros
    X_padded = pad_sequence(X_list, batch_first=True, padding_value=0)
    
    # pad manually if needed
    if X_padded.size(1) < MAX_SEQ_LEN:
        pad_size = MAX_SEQ_LEN - X_padded.size(1)
        zeros = torch.zeros(X_padded.size(0), pad_size)
        X_padded = torch.cat([zeros, X_padded], dim=1) # Padding zleva
        
    y_tensor = torch.tensor(y_list, dtype=torch.float32).unsqueeze(1)
    
    # LSTM input format: (Batch, Seq, Feature) / add 1
    X_padded = X_padded.unsqueeze(-1)
    
    return X_padded, y_tensor

print("creating tensors...")
X_train, y_train = create_dataset('train')
X_val, y_val = create_dataset('val') 
X_test, y_test = create_dataset('test')

print(f"Train shape: {X_train.shape}")
print(f"Test shape:  {X_test.shape}")